# RNN Surrogate Model for Double Pendulum


## 1. Import Data and Configuration

Import Configuration.

In [22]:
import yaml

with open("Configuration/DoublePendulum.yaml", "r") as f:
    config = yaml.load(f, Loader=yaml.FullLoader)

config

{'DoF': 2,
 'PARAMETER': {'m1': 1.2, 'm2': 1.4, 'l1': 1.7, 'l2': 2.1, 'g': 9.8},
 'DATASET': {'NUM_SAMPLES': 80,
  'TIME_STEPS': 2000,
  'SAMPLING_TIME': 0.001,
  'SOLVER_SLICE': 100,
  'RELTOL': 1.01e-08,
  'ABSTOL': 1.01e-08},
 'DATA_LOADER': {'BATCH_SIZE': 16, 'SEQ_LEN': 50},
 'NETWORK': {'NUM_LAYERS': 1, 'HIDDEN_SIZE': 16}}

Import Data.

In [20]:
from scipy.io import loadmat

MatData = loadmat("Data/DoublePendulum.mat")
TimeHistoryData = MatData["Data"]

Convert to sequences.

In [24]:
import numpy as np

InputData = np.zeros((config["DATA_LOADER"]["SEQ_LEN"], 2 * config["DoF"]))
InputData

array([[0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
